In [ ]:
print('Setup complete.')

# Spec-to-Script Demo

## Learning Objectives
- Transform written specifications into working code using AI
- Build iterative development workflows with AI assistance
- Create robust script generation with validation and testing
- Design AI-assisted development patterns for rapid prototyping

## The Challenge: From Idea to Implementation

This demo shows how to go from a written specification to a complete, working script using AI assistance. We'll build a systematic approach for:
1. **Spec Analysis** - Understanding requirements and constraints
2. **Code Generation** - AI-assisted implementation
3. **Validation** - Testing and refinement
4. **Documentation** - Self-documenting code generation

In [ ]:
# Install required packages
!pip install asksageclient pip_system_certs rich click tiktoken

In [ ]:
# ================================
# 🔐 Cell 1 — Load secrets (Colab) + pricing + token utils
# ================================
import os, time, csv
from typing import Optional, Dict
import tiktoken

from google.colab import userdata

ASKSAGE_API_KEY = userdata.get("ASKSAGE_API_KEY")
ASKSAGE_BASE_URL = userdata.get("ASKSAGE_BASE_URL")
ASKSAGE_EMAIL = userdata.get("ASKSAGE_EMAIL")

assert ASKSAGE_API_KEY, "ASKSAGE_API_KEY not provided."
assert ASKSAGE_EMAIL, "ASKSAGE_EMAIL not provided."

print("✓ Secrets loaded")
print("  • EMAIL:", ASKSAGE_EMAIL)
print("  • BASE URL:", ASKSAGE_BASE_URL or "(default)")

# Pricing (USD per 1,000,000 tokens)
PRICES_PER_M = {
    "gpt-5": {"input_per_m": 1.25, "output_per_m": 10.00},
    "gpt-5-mini": {"input_per_m": 0.25, "output_per_m": 2.00},
}

# Tokenizer
enc = tiktoken.get_encoding("o200k_base")

def count_tokens(text: str) -> int:
    return len(enc.encode(text or ""))

def cost_usd(model: str, input_tokens: int, output_tokens: int) -> float:
    if model not in PRICES_PER_M:
        raise ValueError(f"Unknown model: {model}")
    r = PRICES_PER_M[model]
    return (input_tokens / 1_000_000) * r["input_per_m"] + (output_tokens / 1_000_000) * r["output_per_m"]

In [ ]:
# ================================
# 🔧 Cell 2 — Import bootcamp_common and setup AskSage client
# ================================
import sys
sys.path.append('../../../')  # Adjust path to reach bootcamp_common

from bootcamp_common.ask_sage import AskSageClient

# Initialize AskSage client
client = AskSageClient(
    api_key=ASKSAGE_API_KEY,
    base_url=ASKSAGE_BASE_URL
)

print("✓ AskSage client initialized")

In [ ]:
import os
import ast
import subprocess
from typing import Dict, List, Optional, Any
from dataclasses import dataclass
from pathlib import Path

import openai
from rich.console import Console
from rich.panel import Panel
from rich.syntax import Syntax
from rich.table import Table

console = Console()
print("🔧 Spec-to-Script system loading...")

## Sample Specification: File Organizer Tool

Let's work with a realistic specification for a file organization utility.

In [ ]:
sample_spec = """
# File Organizer Tool Specification

## Overview
Create a command-line tool that automatically organizes files in a directory based on their types and dates.

## Requirements
1. **File Type Organization**: Group files by extension (images, documents, videos, etc.)
2. **Date-based Sorting**: Optionally organize by creation/modification date
3. **Safe Operations**: Always confirm before moving files, with undo capability
4. **Configurable Rules**: Allow custom organization rules via config file
5. **Progress Reporting**: Show progress for large operations
6. **Dry Run Mode**: Preview changes without actually moving files

## Command Interface
```
python file_organizer.py [OPTIONS] DIRECTORY

Options:
  --by-type          Organize by file type (default)
  --by-date          Organize by date
  --dry-run          Preview changes without executing
  --config FILE      Use custom configuration file
  --quiet            Suppress progress output
  --help             Show help message
```

## Configuration Format
```yaml
file_types:
  images: ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
  documents: ['.pdf', '.doc', '.docx', '.txt', '.md']
  videos: ['.mp4', '.avi', '.mkv', '.mov']
  archives: ['.zip', '.tar', '.gz', '.rar']

folder_structure:
  by_type: '{type}'
  by_date: '{year}/{month}'
```

## Error Handling
- Handle permission errors gracefully
- Prevent overwriting existing files
- Validate directory paths before processing
- Log all operations for debugging
"""

console.print("📋 [bold blue]Sample Specification Loaded[/bold blue]")
console.print(Panel(sample_spec, title="File Organizer Specification", border_style="blue"))
print("\n🎯 Ready to transform this spec into working code!")

## Spec-to-Script Generator

In [ ]:
@dataclass
class CodeGenerationResult:
    """Result of code generation process"""
    success: bool
    code: str
    dependencies: List[str]
    tests_generated: bool
    validation_passed: bool
    issues: List[str]

class SpecToScriptGenerator:
    """Generate Python scripts from written specifications"""
    
    def __init__(self):
        self.setup_client()
        self.generated_scripts = {}
    
    def setup_client(self):
        """Setup API client with fallback to mock"""
        if os.getenv('OPENAI_API_KEY'):
            try:
                self.client = openai.OpenAI()
                self.has_api = True
                console.print("✅ OpenAI client configured")
            except Exception as e:
                self.has_api = False
                console.print(f"⚠️ Using mock responses: {e}")
        else:
            self.has_api = False
            console.print("💡 No API key found, using mock responses")
    
    def analyze_specification(self, spec: str) -> Dict[str, Any]:
        """Analyze specification to extract key requirements"""
        
        analysis_prompt = f"""Analyze this software specification and extract key information:

{spec}

Extract and structure the following information:
1. Core functionality requirements
2. Input/output specifications
3. Dependencies and libraries needed
4. Error handling requirements
5. Configuration needs
6. Testing considerations

Respond in structured format suitable for code generation."""
        
        if self.has_api:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": analysis_prompt}],
                max_tokens=600,
                temperature=0.3
            )
            analysis = response.choices[0].message.content
        else:
            # Mock analysis
            analysis = """Core functionality:
- File organization by type and date
- CLI interface with multiple options
- Configuration file support
- Dry run and progress reporting

Dependencies: click, pyyaml, pathlib
Testing: Unit tests for file operations, CLI testing
Error handling: Permission errors, file conflicts"""
        
        return {"analysis": analysis}
    
    def generate_code(self, spec: str, analysis: Dict[str, Any]) -> str:
        """Generate Python code from specification"""
        
        generation_prompt = f"""Generate a complete, production-ready Python script based on this specification:

SPECIFICATION:
{spec}

ANALYSIS:
{analysis.get('analysis', '')}

Requirements for the generated code:
1. Complete, runnable Python script
2. Proper error handling and logging
3. Clear documentation and comments
4. Follow Python best practices
5. Include command-line interface
6. Add type hints where appropriate

Generate ONLY the Python code, well-structured and documented."""
        
        if self.has_api:
            response = self.client.chat.completions.create(
                model="gpt-4",  # Use GPT-4 for better code generation
                messages=[{"role": "user", "content": generation_prompt}],
                max_tokens=2000,
                temperature=0.1
            )
            code = response.choices[0].message.content
        else:
            # Mock code generation
            code = '''#!/usr/bin/env python3
"""
File Organizer Tool
Automatically organizes files by type and date.
"""

import os
import shutil
from pathlib import Path
from typing import Dict, List
import click
import yaml

@click.command()
@click.argument('directory', type=click.Path(exists=True))
@click.option('--by-type', is_flag=True, default=True, help='Organize by file type')
@click.option('--dry-run', is_flag=True, help='Preview changes without executing')
def organize_files(directory: str, by_type: bool, dry_run: bool):
    """Organize files in the specified directory."""
    click.echo(f"Organizing files in: {directory}")
    
    if dry_run:
        click.echo("DRY RUN MODE - No files will be moved")
    
    # File type mappings
    file_types = {
        'images': ['.jpg', '.jpeg', '.png', '.gif'],
        'documents': ['.pdf', '.doc', '.docx', '.txt'],
        'videos': ['.mp4', '.avi', '.mkv']
    }
    
    path = Path(directory)
    
    for file_path in path.iterdir():
        if file_path.is_file():
            file_type = get_file_type(file_path.suffix.lower(), file_types)
            target_dir = path / file_type
            
            if not dry_run:
                target_dir.mkdir(exist_ok=True)
                shutil.move(str(file_path), str(target_dir / file_path.name))
                click.echo(f"Moved: {file_path.name} -> {file_type}/")
            else:
                click.echo(f"Would move: {file_path.name} -> {file_type}/")

def get_file_type(extension: str, file_types: Dict[str, List[str]]) -> str:
    """Determine file type based on extension."""
    for type_name, extensions in file_types.items():
        if extension in extensions:
            return type_name
    return 'other'

if __name__ == '__main__':
    organize_files()'''
        
        return code
    
    def validate_code(self, code: str) -> List[str]:
        """Validate generated code for syntax and basic issues"""
        issues = []
        
        try:
            # Check syntax
            ast.parse(code)
        except SyntaxError as e:
            issues.append(f"Syntax error: {e}")
        
        # Basic code quality checks
        if 'import' not in code:
            issues.append("No imports found - may be incomplete")
        
        if 'def ' not in code and 'class ' not in code:
            issues.append("No functions or classes defined")
        
        if '__name__' not in code:
            issues.append("Missing main execution block")
        
        return issues
    
    def generate_tests(self, code: str, spec: str) -> str:
        """Generate basic tests for the code"""
        
        test_prompt = f"""Generate pytest unit tests for this Python code:

{code[:1000]}...

Based on specification requirements:
{spec[:500]}...

Generate comprehensive tests covering:
1. Main functionality
2. Error cases
3. Edge cases
4. CLI interface testing

Return only the test code."""
        
        if self.has_api:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": test_prompt}],
                max_tokens=800,
                temperature=0.2
            )
            tests = response.choices[0].message.content
        else:
            # Mock test generation
            tests = '''import pytest
from pathlib import Path
from file_organizer import get_file_type, organize_files

def test_get_file_type():
    file_types = {'images': ['.jpg', '.png']}
    assert get_file_type('.jpg', file_types) == 'images'
    assert get_file_type('.txt', file_types) == 'other'

def test_organize_files_dry_run(tmp_path):
    # Test dry run functionality
    test_file = tmp_path / "test.jpg"
    test_file.touch()
    
    # This would test the CLI in dry run mode
    pass'''
        
        return tests
    
    def process_specification(self, spec: str, name: str = "generated_script") -> CodeGenerationResult:
        """Complete spec-to-script process"""
        
        console.print(f"\n🔄 [bold blue]Processing Specification: {name}[/bold blue]")
        
        try:
            # Step 1: Analyze specification
            console.print("[yellow]1. Analyzing specification...[/yellow]")
            analysis = self.analyze_specification(spec)
            
            # Step 2: Generate code
            console.print("[yellow]2. Generating code...[/yellow]")
            code = self.generate_code(spec, analysis)
            
            # Step 3: Validate code
            console.print("[yellow]3. Validating code...[/yellow]")
            issues = self.validate_code(code)
            
            # Step 4: Generate tests
            console.print("[yellow]4. Generating tests...[/yellow]")
            tests = self.generate_tests(code, spec)
            
            # Store results
            self.generated_scripts[name] = {
                'code': code,
                'tests': tests,
                'analysis': analysis
            }
            
            result = CodeGenerationResult(
                success=len(issues) == 0,
                code=code,
                dependencies=['click', 'pyyaml'],
                tests_generated=len(tests) > 50,
                validation_passed=len(issues) == 0,
                issues=issues
            )
            
            return result
            
        except Exception as e:
            return CodeGenerationResult(
                success=False,
                code="",
                dependencies=[],
                tests_generated=False,
                validation_passed=False,
                issues=[str(e)]
            )

# Initialize generator
generator = SpecToScriptGenerator()
print("🤖 Spec-to-Script generator ready!")

## Demo: Generate File Organizer Script

In [ ]:
# Generate script from specification
console.print("\n🚀 [bold blue]Spec-to-Script Generation Demo[/bold blue]")

result = generator.process_specification(sample_spec, "file_organizer")

# Display results
if result.success:
    console.print("\n[green]✅ Code generation successful![/green]")
    
    # Show generated code
    console.print("\n[yellow]Generated Code:[/yellow]")
    syntax = Syntax(result.code, "python", theme="monokai", line_numbers=True)
    console.print(Panel(syntax, title="file_organizer.py", border_style="green"))
    
    # Show summary
    summary_table = Table(title="Generation Summary")
    summary_table.add_column("Aspect")
    summary_table.add_column("Status")
    summary_table.add_column("Details")
    
    summary_table.add_row(
        "Code Generation",
        "✅ Success" if result.success else "❌ Failed",
        f"{len(result.code)} characters"
    )
    
    summary_table.add_row(
        "Syntax Validation",
        "✅ Passed" if result.validation_passed else "❌ Failed",
        f"{len(result.issues)} issues found" if result.issues else "No issues"
    )
    
    summary_table.add_row(
        "Test Generation",
        "✅ Generated" if result.tests_generated else "❌ Failed",
        "Unit tests created"
    )
    
    summary_table.add_row(
        "Dependencies",
        "📦 Identified",
        ", ".join(result.dependencies)
    )
    
    console.print(summary_table)
    
else:
    console.print("\n[red]❌ Code generation failed![/red]")
    for issue in result.issues:
        console.print(f"[red]• {issue}[/red]")

print("\n🎯 Spec-to-Script generation complete!")

## Key Takeaways: AI-Assisted Development

### 🎯 **Effective Spec-to-Script Process**

1. **Clear Specifications**: Well-defined requirements lead to better code
2. **Iterative Refinement**: Analyze → Generate → Validate → Improve
3. **Built-in Validation**: Always check syntax and basic quality
4. **Test Generation**: Automatically create test scaffolding
5. **Dependency Detection**: Identify required libraries and imports

### 🔧 **Best Practices for AI Code Generation**

- **Structured Prompts**: Use clear templates for consistent results
- **Context Management**: Provide enough detail without overwhelming
- **Quality Gates**: Implement validation at each step
- **Human Review**: AI generates, humans verify and refine
- **Version Control**: Track generated code like any other code

### 🚀 **Production Considerations**

- **Security Review**: Generated code needs security assessment
- **Performance Testing**: Validate efficiency of generated solutions
- **Integration Testing**: Ensure compatibility with existing systems
- **Documentation**: Enhance AI-generated docs with human insight
- **Maintenance**: Plan for ongoing updates and improvements

## Next: Package Refactor Demo

Ready to see how AI can help restructure and improve existing codebases?